In [1]:
from lohrasb.best_estimator import BaseModel
from optuna.pruners import HyperbandPruner
from optuna.samplers._tpe.sampler import TPESampler
from sklearn.model_selection import KFold,train_test_split
import pandas as pd
import optuna
from sklearn.pipeline import Pipeline
from feature_engine.imputation import (
    CategoricalImputer,
    MeanMedianImputer
    )
from category_encoders import OrdinalEncoder
from sklearn.metrics import mean_absolute_error,r2_score
from interpret.glassbox import ExplainableBoostingRegressor
from interpret import show


2023-02-21 20:42:30,980 :: matplotlib :: matplotlib data path: /Users/hjavedani/Documents/Lohrasb/.venv/lib/python3.10/site-packages/matplotlib/mpl-data
2023-02-21 20:42:30,985 :: matplotlib :: CONFIGDIR=/Users/hjavedani/.matplotlib
2023-02-21 20:42:30,988 :: matplotlib :: interactive is False
2023-02-21 20:42:30,990 :: matplotlib :: platform is darwin
2023-02-21 20:42:32,246 :: matplotlib :: CACHEDIR=/Users/hjavedani/.matplotlib
2023-02-21 20:42:32,248 :: matplotlib.font_manager :: Using fontManager instance from /Users/hjavedani/.matplotlib/fontlist-v330.json


#### Example 2 :Computer Hardware Data Set (a regression problem)
  
https://archive.ics.uci.edu/ml/datasets/Computer+Hardware

#### Part 1: Use BestModel in sklearn pipeline


In [2]:
urldata= "https://archive.ics.uci.edu/ml/machine-learning-databases/cpu-performance/machine.data"
# column names
col_names=[
    "vendor name",
    "Model Name",
    "MYCT",
    "MMIN",
    "MMAX",
    "CACH",
    "CHMIN",
    "CHMAX",
    "PRP"
]
# read data
data = pd.read_csv(urldata,header=None,names=col_names,sep=',')
data

,vendor name,Model Name,MYCT,MMIN,MMAX,CACH,CHMIN,CHMAX,PRP
adviser,32/60,125,256,6000,256,16,128,198,199
amdahl,470v/7,29,8000,32000,32,8,32,269,253
amdahl,470v/7a,29,8000,32000,32,8,32,220,253
amdahl,470v/7b,29,8000,32000,32,8,32,172,253
amdahl,470v/7c,29,8000,16000,32,8,16,132,132
...,...,...,...,...,...,...,...,...,...
sperry,80/8,124,1000,8000,0,1,8,42,37
sperry,90/80-model-3,98,1000,8000,32,2,8,46,50
sratus,32,125,2000,8000,0,2,14,52,41
wang,vs-100,480,512,8000,32,0,0,67,47


#### Train test split

In [3]:
X = data.loc[:, data.columns != "PRP"]
y = data.loc[:, data.columns == "PRP"]
y = y.values.ravel()


X_train, X_test, y_train, y_test =train_test_split(X, y, test_size=0.33, random_state=42)


#### Find feature types for later use

In [4]:
int_cols =  X_train.select_dtypes(include=['int']).columns.tolist()
float_cols =  X_train.select_dtypes(include=['float']).columns.tolist()
cat_cols =  X_train.select_dtypes(include=['object']).columns.tolist()


####  Define estimator and set its arguments  

In [5]:
estimator = ExplainableBoostingRegressor()
estimator_params= {
                    "learning_rate":[0.01,0.1], 
                    "validation_size":[0.15,0.20],
                    "early_stopping_rounds":[50,200], 
                    "early_stopping_tolerance":[0.00001,0.001], 
                    "max_rounds":[5000,20000], 
                    "min_samples_leaf":[2,5], 
                    "max_leaves":[2,5]  
                      }


In [6]:
obj = BaseModel().optimize_by_optuna(
            estimator=estimator,
            estimator_params=estimator_params,
            fit_params=None,
            measure_of_accuracy="mean_absolute_error(y_true, y_pred, multioutput='uniform_average')",
            with_stratified=False,
            test_size=.3,
            verbose=3,
            n_jobs=-1,
            random_state=42,
            # optuna params
            # optuna study init params
            study=optuna.create_study(
                storage=None,
                sampler=TPESampler(),
                pruner=HyperbandPruner(),
                study_name=None,
                direction="minimize",
                load_if_exists=False,
                directions=None,
            ),
            # optuna optimization params
            study_optimize_objective=None,
            study_optimize_objective_n_trials=100,
            study_optimize_objective_timeout=600,
            study_optimize_n_jobs=-1,
            study_optimize_catch=(),
            study_optimize_callbacks=None,
            study_optimize_gc_after_trial=False,
            study_optimize_show_progress_bar=False,
        )

[I 2023-02-21 20:42:33,032] A new study created in memory with name: no-name-6b1583af-5ffc-4166-bbe5-3f562c08d0be


#### Build sklearn pipeline

In [7]:


pipeline =Pipeline([
            # int missing values imputers
            ('intimputer', MeanMedianImputer(
                imputation_method='median', variables=int_cols)),
            # category missing values imputers
            ('catimputer', CategoricalImputer(variables=cat_cols)),
            #
            ('catencoder', OrdinalEncoder()),
            # regression model 
            ('obj', obj),


 ])



#### Run Pipeline

In [8]:
pipeline.fit(X_train,y_train)
y_pred = pipeline.predict(X_test)


2023-02-21 20:42:33,119 :: interpret.utils._native :: EBM lib loading.
2023-02-21 20:42:33,122 :: interpret.utils._native :: EBM lib loading.
2023-02-21 20:42:33,125 :: interpret.utils._native :: EBM lib loading.
2023-02-21 20:42:33,128 :: interpret.utils._native :: EBM lib loading.
2023-02-21 20:42:33,130 :: interpret.utils._native :: EBM lib loading.
2023-02-21 20:42:33,131 :: interpret.utils._native :: EBM lib loading.
2023-02-21 20:42:33,131 :: interpret.utils._native :: Loading native on Darwin | debug = False
2023-02-21 20:42:33,134 :: interpret.utils._native :: EBM lib loading.
2023-02-21 20:42:33,136 :: interpret.utils._native :: EBM lib loading.
2023-02-21 20:42:33,137 :: interpret.utils._native :: Loading native on Darwin | debug = False
2023-02-21 20:42:33,138 :: interpret.utils._native :: Loading native on Darwin | debug = False
2023-02-21 20:42:33,139 :: interpret.utils._native :: Loading native on Darwin | debug = False
2023-02-21 20:42:33,140 :: interpret.utils._native :

/Users/hjavedani/Documents/Lohrasb/.venv/lib/python3.10/site-packages/optuna/study/study.py:393: FutureWarning: `n_jobs` argument has been deprecated in v2.7.0. This feature will be removed in v4.0.0. See https://github.com/optuna/optuna/releases/tag/v2.7.0.
  warnings.warn(


2023-02-21 20:42:33,244 :: interpret.utils._binning :: Creating native dataset
2023-02-21 20:42:33,247 :: interpret.utils._binning :: Creating native dataset
2023-02-21 20:42:33,249 :: interpret.utils._binning :: Creating native dataset
2023-02-21 20:42:33,253 :: interpret.utils._binning :: Creating native dataset
2023-02-21 20:42:33,255 :: interpret.utils._binning :: Creating native dataset
2023-02-21 20:42:33,258 :: interpret.utils._binning :: Creating native dataset
2023-02-21 20:42:40,641 :: interpret.glassbox.ebm.bin :: eval_terms
2023-02-21 20:42:40,648 :: interpret.glassbox.ebm.bin :: eval_terms
2023-02-21 20:42:40,651 :: interpret.glassbox.ebm.bin :: eval_terms
2023-02-21 20:42:40,653 :: interpret.glassbox.ebm.bin :: eval_terms
2023-02-21 20:42:40,656 :: interpret.glassbox.ebm.bin :: eval_terms
2023-02-21 20:42:40,657 :: interpret.glassbox.ebm.bin :: eval_terms
2023-02-21 20:42:40,660 :: interpret.glassbox.ebm.bin :: eval_terms
2023-02-21 20:42:40,664 :: interpret.glassbox.ebm.

[I 2023-02-21 20:42:41,039] Trial 5 finished with value: 16.127718993135634 and parameters: {'learning_rate': 0.028370999072418572, 'validation_size': 0.1994533179448228, 'early_stopping_rounds': 61, 'early_stopping_tolerance': 0.0005785155693317803, 'max_rounds': 17247, 'min_samples_leaf': 4, 'max_leaves': 3}. Best is trial 5 with value: 16.127718993135634.


2023-02-21 20:42:41,055 :: interpret.utils._binning :: Creating native dataset
2023-02-21 20:42:41,597 :: interpret.glassbox.ebm.bin :: eval_terms
2023-02-21 20:42:41,604 :: interpret.glassbox.ebm.bin :: eval_terms
2023-02-21 20:42:41,610 :: interpret.glassbox.ebm.bin :: eval_terms
2023-02-21 20:42:41,613 :: interpret.glassbox.ebm.bin :: eval_terms
2023-02-21 20:42:41,617 :: interpret.glassbox.ebm.bin :: eval_terms
2023-02-21 20:42:41,621 :: interpret.glassbox.ebm.bin :: eval_terms
2023-02-21 20:42:41,624 :: interpret.glassbox.ebm.bin :: eval_terms
2023-02-21 20:42:41,627 :: interpret.glassbox.ebm.bin :: eval_terms
2023-02-21 20:42:41,631 :: interpret.utils._binning :: Creating native dataset
2023-02-21 20:42:41,636 :: interpret.glassbox.ebm.ebm :: Estimating with FAST
2023-02-21 20:42:41,829 :: interpret.glassbox.ebm.bin :: eval_terms
2023-02-21 20:42:41,846 :: interpret.glassbox.ebm.bin :: eval_terms


...

2023-02-21 20:43:33,816 :: interpret.glassbox.ebm.bin :: eval_terms
2023-02-21 20:43:33,826 :: interpret.glassbox.ebm.bin :: eval_terms


[I 2023-02-21 20:43:33,832] Trial 99 finished with value: 12.413506228251295 and parameters: {'learning_rate': 0.07650446528794591, 'validation_size': 0.1511693718216853, 'early_stopping_rounds': 86, 'early_stopping_tolerance': 0.000534411620475753, 'max_rounds': 18960, 'min_samples_leaf': 2, 'max_leaves': 2}. Best is trial 1 with value: 11.721424841576487.


2023-02-21 20:43:34,188 :: interpret.glassbox.ebm.bin :: eval_terms
2023-02-21 20:43:34,199 :: interpret.glassbox.ebm.bin :: eval_terms


[I 2023-02-21 20:43:34,202] Trial 97 finished with value: 12.570293677483841 and parameters: {'learning_rate': 0.07603290793260943, 'validation_size': 0.15315156449060469, 'early_stopping_rounds': 84, 'early_stopping_tolerance': 0.0005022009190367052, 'max_rounds': 13958, 'min_samples_leaf': 2, 'max_leaves': 2}. Best is trial 1 with value: 11.721424841576487.


2023-02-21 20:43:34,212 :: interpret.utils._binning :: Creating native dataset
2023-02-21 20:43:34,348 :: interpret.glassbox.ebm.bin :: eval_terms
2023-02-21 20:43:34,350 :: interpret.glassbox.ebm.bin :: eval_terms
2023-02-21 20:43:34,352 :: interpret.glassbox.ebm.bin :: eval_terms
2023-02-21 20:43:34,354 :: interpret.glassbox.ebm.bin :: eval_terms
2023-02-21 20:43:34,356 :: interpret.glassbox.ebm.bin :: eval_terms
2023-02-21 20:43:34,359 :: interpret.glassbox.ebm.bin :: eval_terms
2023-02-21 20:43:34,362 :: interpret.glassbox.ebm.bin :: eval_terms
2023-02-21 20:43:34,364 :: interpret.glassbox.ebm.bin :: eval_terms
2023-02-21 20:43:34,366 :: interpret.utils._binning :: Creating native dataset
2023-02-21 20:43:34,370 :: interpret.glassbox.ebm.ebm :: Estimating with FAST
2023-02-21 20:43:35,472 :: interpret.glassbox.ebm.bin :: eval_terms
2023-02-21 20:43:35,488 :: interpret.glassbox.ebm.bin :: eval_terms


#### Check performance of the pipeline

In [9]:
print('r2 score : ')
print(r2_score(y_test,y_pred))


r2 score : 
0.5375864927331135


#### Part 2:  Use BestModel as a standalone estimator 

In [10]:
X_train, X_test, y_train, y_test =train_test_split(X, y, test_size=0.33, random_state=42)


#### Transform features to make them ready for model input

In [11]:
transform_pipeline =Pipeline([
            # int missing values imputers
            ('intimputer', MeanMedianImputer(
                imputation_method='median', variables=int_cols)),
            # category missing values imputers
            ('catimputer', CategoricalImputer(variables=cat_cols)),
            #
            ('catencoder', OrdinalEncoder()),
            # classification model

 ])

#### Transform X_train and X_test

In [12]:
X_train=transform_pipeline.fit_transform(X_train,y_train)
X_test=transform_pipeline.transform(X_test)


#### Train model and predict

In [13]:
obj.fit(X_train,y_train)
y_pred = obj.predict(X_test)

2023-02-21 20:43:35,869 :: interpret.utils._binning :: Creating native dataset
2023-02-21 20:43:35,875 :: interpret.utils._binning :: Creating native dataset
2023-02-21 20:43:35,885 :: interpret.utils._binning :: Creating native dataset
2023-02-21 20:43:35,886 :: interpret.utils._binning :: Creating native dataset


/Users/hjavedani/Documents/Lohrasb/.venv/lib/python3.10/site-packages/optuna/study/study.py:393: FutureWarning: `n_jobs` argument has been deprecated in v2.7.0. This feature will be removed in v4.0.0. See https://github.com/optuna/optuna/releases/tag/v2.7.0.
  warnings.warn(


2023-02-21 20:43:35,890 :: interpret.utils._binning :: Creating native dataset
2023-02-21 20:43:35,893 :: interpret.utils._binning :: Creating native dataset
2023-02-21 20:43:35,894 :: interpret.utils._binning :: Creating native dataset
2023-02-21 20:43:35,895 :: interpret.utils._binning :: Creating native dataset
2023-02-21 20:43:36,041 :: interpret.glassbox.ebm.bin :: eval_terms
2023-02-21 20:43:36,044 :: interpret.glassbox.ebm.bin :: eval_terms
2023-02-21 20:43:36,047 :: interpret.glassbox.ebm.bin :: eval_terms
2023-02-21 20:43:36,050 :: interpret.glassbox.ebm.bin :: eval_terms
2023-02-21 20:43:36,053 :: interpret.glassbox.ebm.bin :: eval_terms
2023-02-21 20:43:36,058 :: interpret.glassbox.ebm.bin :: eval_terms
2023-02-21 20:43:36,062 :: interpret.glassbox.ebm.bin :: eval_terms
2023-02-21 20:43:36,066 :: interpret.glassbox.ebm.bin :: eval_terms
2023-02-21 20:43:36,070 :: interpret.utils._binning :: Creating native dataset
2023-02-21 20:43:36,074 :: interpret.glassbox.ebm.ebm :: Esti

[I 2023-02-21 20:43:37,595] Trial 104 finished with value: 13.191656545184102 and parameters: {'learning_rate': 0.07289259509556328, 'validation_size': 0.1503872948311201, 'early_stopping_rounds': 185, 'early_stopping_tolerance': 0.0003474951462319508, 'max_rounds': 10998, 'min_samples_leaf': 2, 'max_leaves': 3}. Best is trial 1 with value: 11.721424841576487.


2023-02-21 20:43:37,624 :: interpret.glassbox.ebm.bin :: eval_terms
2023-02-21 20:43:37,641 :: interpret.glassbox.ebm.bin :: eval_terms


...

2023-02-21 20:44:23,987 :: interpret.glassbox.ebm.bin :: eval_terms
2023-02-21 20:44:23,999 :: interpret.glassbox.ebm.bin :: eval_terms


[I 2023-02-21 20:44:24,002] Trial 198 finished with value: 11.840990220139274 and parameters: {'learning_rate': 0.09977092181181958, 'validation_size': 0.15913209047234922, 'early_stopping_rounds': 109, 'early_stopping_tolerance': 0.00012215452353886452, 'max_rounds': 15279, 'min_samples_leaf': 2, 'max_leaves': 2}. Best is trial 136 with value: 11.717498291897297.


2023-02-21 20:44:24,149 :: interpret.glassbox.ebm.bin :: eval_terms
2023-02-21 20:44:24,158 :: interpret.glassbox.ebm.bin :: eval_terms


[I 2023-02-21 20:44:24,162] Trial 199 finished with value: 11.847504716391132 and parameters: {'learning_rate': 0.09990829269631057, 'validation_size': 0.15877039784293784, 'early_stopping_rounds': 110, 'early_stopping_tolerance': 0.00011740120810099084, 'max_rounds': 15834, 'min_samples_leaf': 2, 'max_leaves': 2}. Best is trial 136 with value: 11.717498291897297.


2023-02-21 20:44:24,172 :: interpret.utils._binning :: Creating native dataset
2023-02-21 20:44:24,305 :: interpret.glassbox.ebm.bin :: eval_terms
2023-02-21 20:44:24,307 :: interpret.glassbox.ebm.bin :: eval_terms
2023-02-21 20:44:24,309 :: interpret.glassbox.ebm.bin :: eval_terms
2023-02-21 20:44:24,312 :: interpret.glassbox.ebm.bin :: eval_terms
2023-02-21 20:44:24,315 :: interpret.glassbox.ebm.bin :: eval_terms
2023-02-21 20:44:24,317 :: interpret.glassbox.ebm.bin :: eval_terms
2023-02-21 20:44:24,320 :: interpret.glassbox.ebm.bin :: eval_terms
2023-02-21 20:44:24,322 :: interpret.glassbox.ebm.bin :: eval_terms
2023-02-21 20:44:24,325 :: interpret.utils._binning :: Creating native dataset
2023-02-21 20:44:24,330 :: interpret.glassbox.ebm.ebm :: Estimating with FAST
2023-02-21 20:44:25,186 :: interpret.glassbox.ebm.bin :: eval_terms
2023-02-21 20:44:25,198 :: interpret.glassbox.ebm.bin :: eval_terms


#### Check performance of the model

In [14]:
print('r2 score : ')
print(r2_score(y_test,y_pred))
print('mean_absolute_error : ')
print(mean_absolute_error(y_test,y_pred))


r2 score : 
0.5767150008250387
mean_absolute_error : 
41.90232547571242


In [15]:
ebm=obj.get_best_estimator()
print(ebm)

ExplainableBoostingRegressor(early_stopping_rounds=136,
                             early_stopping_tolerance=0.00013710838471780898,
                             learning_rate=0.09740051475479909, max_leaves=2,
                             max_rounds=15716,
                             validation_size=0.15880868368853937)


In [16]:
obj.best_estimator

ExplainableBoostingRegressor(early_stopping_rounds=136,
                             early_stopping_tolerance=0.00013710838471780898,
                             learning_rate=0.09740051475479909, max_leaves=2,
                             max_rounds=15716,
                             validation_size=0.15880868368853937)

#### Get fitted randomized search object and its attributes

In [17]:
OptunaObj = obj.get_optimized_object()
OptunaObj.trials

[FrozenTrial(number=0, values=[14.511573530646379], datetime_start=datetime.datetime(2023, 2, 21, 20, 42, 33, 118432), datetime_complete=datetime.datetime(2023, 2, 21, 20, 42, 41, 850140), params={'learning_rate': 0.021463880765159656, 'validation_size': 0.16540686977778507, 'early_stopping_rounds': 127, 'early_stopping_tolerance': 0.0005243365242405249, 'max_rounds': 12211, 'min_samples_leaf': 4, 'max_leaves': 4}, distributions={'learning_rate': UniformDistribution(high=0.1, low=0.01), 'validation_size': UniformDistribution(high=0.2, low=0.15), 'early_stopping_rounds': IntUniformDistribution(high=200, low=50, step=1), 'early_stopping_tolerance': UniformDistribution(high=0.001, low=1e-05), 'max_rounds': IntUniformDistribution(high=20000, low=5000, step=1), 'min_samples_leaf': IntUniformDistribution(high=5, low=2, step=1), 'max_leaves': IntUniformDistribution(high=5, low=2, step=1)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=0, state=TrialState.COMPLETE, value=Non

#### This visualizations provided is for global explanations.



In [18]:
ebm_global = ebm.explain_global()
show(ebm_global)


2023-02-21 20:44:25,496 :: interpret.provider.visualize :: Detected non-cloud environment.
2023-02-21 20:44:26,359 :: urllib3.connectionpool :: Starting new HTTP connection (1): 127.0.0.1:7001
2023-02-21 20:44:26,368 :: interpret.visual.udash :: Generating mini dash
2023-02-21 20:44:26,389 :: interpret.visual.udash :: Generated mini dash


<!-- http://127.0.0.1:7001/5197223520/ -->

#### This visualizations provided is for local explanations.


In [19]:
ebm_local = ebm.explain_local(X_test[:5], y_test[:5])
show(ebm_local)


2023-02-21 20:44:26,452 :: interpret.utils._binning :: Unifying data
2023-02-21 20:44:26,455 :: interpret.glassbox.ebm.bin :: eval_terms
2023-02-21 20:44:26,459 :: interpret.glassbox.ebm.bin :: eval_terms
2023-02-21 20:44:26,552 :: urllib3.connectionpool :: Starting new HTTP connection (1): 127.0.0.1:7001
2023-02-21 20:44:26,562 :: urllib3.connectionpool :: http://127.0.0.1:7001 "GET / HTTP/1.1" 200 2067
2023-02-21 20:44:26,564 :: interpret.visual.udash :: Generating mini dash
2023-02-21 20:44:26,575 :: interpret.visual.udash :: Generated mini dash


<!-- http://127.0.0.1:7001/5208628560/ -->

2023-02-21 20:44:26,966 :: interpret.visual.udash :: No overall plot to display: -1|ExplainableBoostingRegressor_1
